# Business Understanding

Our company is expanding by purchasing and operating airplanes for commercial and private enterprises. In this analysis, I determine which aircraft are the lowest risk for the company to start this new business endeavor. I use the fatal injuries rate as a meaure of safety and investigate the effect the make and model of the aircraft, the number of engines, and the location have on the fatal injuries rate.

I investigate the following three business questions:
1. What number of engines is least likely to result in a fatal injury?
2. What is the safest aircraft model?
3. What is the safest location to operate out of?

# Data Understanding

The [Aviation Accident Database & Synopses, up to 2023](https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses/data) from the NTSB (National Transportation Safety Board) aviation accident database contains information about ​civil aviation accidents and selected incidents that occurred between 1962 and 2023 within the United States, its territories and possessions, and in international waters. From this database, I analyze the fatal injuries rate, the make/model of the aircrafts, the number of engines on the aircrafts, and the location of the incidents to determine the  safest aircraft to purchase.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
aviation_df = pd.read_csv('AviationData.csv', encoding='latin')

/var/folders/pn/v9tvzf9s5ms7zcp53821ct0m0000gn/T/ipykernel_16668/3544946329.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  aviation_df = pd.read_csv('AviationData.csv', encoding='latin')


## Data Preparation

The dataset has approxiately 89,000 accidents and incidents. All but 4 columns have varyign numbers of null values. There are approximately 28,000 unique locations and 8,000 unique makes. These will have to be greatly narrowed down to determine 3 recommended locations and 3 recommended aircrafts.

In [3]:
aviation_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [4]:
aviation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [5]:
# Count of how many null values are in each column
aviation_df.isnull().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: i

In [6]:
# Number of options for cities
aviation_df['Location'].nunique()

27758

In [7]:
aviation_df['Make'].nunique()

8237

In [8]:
# Print a count of the top 5 values in each column
for col in aviation_df.columns:
    print(col, '\n', aviation_df[col].value_counts(normalize=True).head(), '\n\n')

Event.Id 
 Event.Id
20001212X19172    0.000034
20001214X45071    0.000034
20220730105623    0.000022
20051213X01965    0.000022
20001212X16765    0.000022
Name: proportion, dtype: float64 


Investigation.Type 
 Investigation.Type
Accident    0.956418
Incident    0.043582
Name: proportion, dtype: float64 


Accident.Number 
 Accident.Number
CEN22LA149    0.000022
WPR23LA041    0.000022
WPR23LA045    0.000022
DCA22WA214    0.000022
DCA22WA089    0.000022
Name: proportion, dtype: float64 


Event.Date 
 Event.Date
1984-06-30    0.000281
1982-05-16    0.000281
2000-07-08    0.000281
1983-08-05    0.000270
1984-08-25    0.000270
Name: proportion, dtype: float64 


Location 
 Location
ANCHORAGE, AK      0.004885
MIAMI, FL          0.002251
ALBUQUERQUE, NM    0.002206
HOUSTON, TX        0.002173
CHICAGO, IL        0.002071
Name: proportion, dtype: float64 


Country 
 Country
United States     0.927647
Brazil            0.004218
Canada            0.004049
Mexico            0.004038
United Ki

## Data Cleaning

Before beginning any type of analysis, I must clean the dataset. 
1. I check for **duplicates** (there are none).
2. I **drop columns** that will not be useful to my analysis.
3. I standardize the **column names** and **make names**. 
4. I **create 3 more columns** that will be useful during the analysis: Total_Passengers, Fatal_Injuries_Rate, Uninjured_Rate.
5. I **drop na's** from columns that include number or rates of injuries, fatalities or uninjured because without those numbers, I will not be able to determine safety.

In [9]:
# Make a copy of the dataset to clean
aviation_clean = aviation_df.copy()

In [10]:
# No duplicates
duplicates = aviation_clean[aviation_clean.duplicated()]
len(duplicates)

0

In [ ]:
# Clean column names by replacing '.' in column names with '_'
aviation_clean.columns = aviation_clean.columns.map(lambda x: x.replace('.', '_',).title())

In [ ]:
# Rewrite makes in title case
aviation_clean['Make'] = aviation_clean['Make'].map(lambda x: str(x).title())

In [11]:
aviation_clean.drop(columns=['Latitude', 'Longitude', 'Schedule', 'Air_Carrier', 'Aircraft_Category', 'Far_Description'], inplace=True)

KeyError: "['Air_Carrier', 'Aircraft_Category', 'Far_Description'] not found in axis"

In [ ]:
# Create a column with Total Number of Passengers
aviation_clean['Total_Passengers'] = aviation_clean['Total_Fatal_Injuries'] + aviation_clean['Total_Serious_Injuries'] \
    + aviation_clean['Total_Minor_Injuries'] + aviation_clean['Total_Uninjured']

# Create column with proportion of fatal injuries per accident
aviation_clean['Fatal_Injuries_Rate'] = aviation_clean['Total_Fatal_Injuries'] / aviation_clean['Total_Passengers']

# Create column with proportion of uninjured per accident
aviation_clean['Uninjured_Rate'] = aviation_clean['Total_Uninjured'] / aviation_clean['Total_Passengers']

In [ ]:
# NaNs are caused by diving by 0
aviation_clean[aviation_clean['Fatal_Injuries_Rate'].isna()].sample(20)

In [ ]:
# Clean NaNs from Total Fatal Injuries and Total Uninjured
# I will remove all of the NaNs because without injury and fatality reports, I will not be able to determine safey
aviation_clean.dropna(subset=['Total_Fatal_Injuries', 'Total_Serious_Injuries', 'Total_Minor_Injuries', 'Total_Uninjured', \
                              'Fatal_Injuries_Rate', 'Uninjured_Rate'], inplace=True)

I subset my data into 3 different dataframes in order to answer our 3 business questions: 
1. **us_flights**: This is a subset of domestic flights, which makes up 93% of the data. I drop na's and split Location into City and State.
3. **top_4_models**: This is a subset of the 4 most common manufacturers according to [Forbes](https://www.forbes.com/sites/geoffwhitmore/2019/08/16/what-is-the-safest-airplane-to-fly/?sh=1fee35415c72). I standardize the names of the 4 manufacturers.
4. **engine_data**: This is a subset of all the non-null rows of engine number.

In [ ]:
# Narrow the data down to the United States
us_flights = aviation_clean[aviation_clean['Country']=='United States']
us_flights.info()

In [ ]:
# Find out what some missing values in the Location column are
us_flights.loc[us_flights['Location'].isna()]

In [ ]:
# I removed the remaining NaNs, as there are only 4 out of approximately 82,000.
us_flights['Location'].dropna(inplace=True)

In [ ]:
# Create 2 new columns, 'City' and 'State' and drop 'Location'
us_flights['City'] = us_flights['Location'].map(lambda x: str(x).title()[:-4])
us_flights['State'] = us_flights['Location'].map(lambda x: str(x)[-2:])
us_flights.drop(columns=['Location'])

In [ ]:
# Create new dataframe with top 4 models. I chose the most common manufacturers according to Forbes.
top_4_makes = us_flights[us_flights['Make'].str.contains('|'.join(['Boeing', 'Airbus', 'Bombardier', 'Embraer']))]
top_4_makes.head()

In [ ]:
top_4_makes['Make'].value_counts()

In [ ]:
# Create a function to standardize the aircraft names

def standardize_aircraft_name(name):
    if 'Boeing' in name:
        new_name = name.replace(name, 'Boeing')
    elif 'Airbus' in name:
        new_name = name.replace(name, 'Airbus')
    elif 'Embraer' in name:
        new_name = name.replace(name, 'Embraer')
    elif 'Bombardier' in name:
        new_name = name.replace(name, 'Bombardier')
    else:
        new_name = name
    return new_name

In [ ]:
# I noticed there were variations of Make names, so I wanted to standardize the names
top_4_makes['Make'] = top_4_makes['Make'].map(lambda x: standardize_aircraft_name(x))

In [ ]:
top_4_makes['Make'].value_counts()

In [ ]:
top_4_makes.info()

In [ ]:
# Clean missing values in 'Number of Engines' 

engine_data = aviation_clean[aviation_clean['Number_Of_Engines'].notnull()]

In [ ]:
# Export the clean data frame using df.to_csv() and upload to tableau to produce visuals

top_4_makes.to_csv('top_4_makes.csv')

# engine_data will be uploaded separately as it is a much smaller dataset. I want to keep more values to answer the other 2 business questions.
engine_data.to_csv('engine_data.csv');

# Exploratory Data Analysis

In [ ]:
sns.set(style='whitegrid', context='notebook')

## Make Analysis

I began with comparing the top 4 aircraft makes with the **average number of injuries** per accident. Bombardier has the *least* average number ofi njuries, while the average total fatal injuries is greater than Airbus, but comparable to Boeing and Embraer. Boeing has the *highest* average number of minor, serious, and fatal injuries.

In [ ]:
# TO DO: ADD VALUES OF EACH BAR SECTION
ax = top_4_makes.groupby(['Make'])[['Total_Fatal_Injuries', 'Total_Minor_Injuries', 'Total_Serious_Injuries']]\
    .mean().sort_values(by='Total_Fatal_Injuries', ascending=True).head().plot(kind='bar', stacked=True)
ax.set_title('Average Total Injuries By Make')
ax.set_xlabel('Aircraft Make')
ax.set_ylabel('Average Number of Injuries');

Next, I analyzed the **fatality rates** by make. Boeing and Bombardier hold the *lowest* fatality rates out of the top for manufacturers, while Embraer has the *highest* fatality rate among the four.

In [ ]:
ax = sns.barplot(data=top_4_makes, x='Make', y='Fatal_Injuries_Rate')
ax.bar_label(ax.containers[0], fontsize=10);
ax.set(xlabel='Make', ylabel='Fatalality Rate', title='Fatality Rate By Make');

Lastly, I analyzed the **uninjured rate** by make. Bombardier came out on *top*, with a 93% uninjured rate. The remaining three manufacturers are quite comparable, ranging from 86% to 88%.

In [ ]:
ax = sns.barplot(data=top_4_makes, x='Make', y='Uninjured_Rate')
ax.bar_label(ax.containers[0], fontsize=10);
ax.set(xlabel='Make', ylabel='Uninjured Rate', title='Uninjured Rate By Make');

In [ ]:
## DELETE
top_4_makes[top_4_makes['Make']=='Bombardier'].groupby(['Model'])[['Total_Uninjured']]\
    .mean().sort_values(by='Total_Uninjured', ascending=True).plot(kind='bar', stacked=True)

In [ ]:
# DELETE
top_4_makes[top_4_makes['Make']=='Bombardier'].groupby(['Model'])[['Fatal_Injuries_Rate']]\
    .mean().sort_values(by='Fatal_Injuries_Rate', ascending=True).plot(kind='bar', stacked=True)

In [ ]:
# DELETE
top_4_makes.groupby(['Model'])['Uninjured_Rate'].mean().sort_values(ascending=False).head().plot(kind='bar')

## Number of Engines Analysis

Aircrafts with **3 engines** hold both the *highest* uninjured rate and the *lowest* fatality rate. Aircrafts with 

In [ ]:
engine_data.groupby(['Number_Of_Engines'])[['Fatal_Injuries_Rate', 'Uninjured_Rate']]\
    .mean().sort_values(by='Uninjured_Rate', ascending=True).plot(kind='bar', stacked=False);

In [ ]:
engine_data['Model'][engine_data['Number_Of_Engines']==8]

In [ ]:
sns.scatterplot(data=engine_data, x='Uninjured_Rate', y='Number_Of_Engines')

In [ ]:
engine_data.plot(x='Fatal_Injuries_Rate', y='Number_Of_Engines', kind='scatter')

In [ ]:
# Bombardier aircrafts only come with 1 or 2 engines. 2 engines are safer than one, according to the data, so I recommend we purchase 
# an aircraft Bombardier aircraft with 2 engines.
make_options = top_4_makes['Model'][(top_4_makes['Make']=='Bombardier') & (top_4_makes['Number_Of_Engines']==2)]
make_options

## Location Analysis

In [ ]:
# Geographical of Location vs. Total Fatal Injuries

# Conclusions

## Limitations

There are various limitations in this analysis.

1. I could not compute the **proportion of accidents to total number of flights**, as this data includes only records of accidents or incidents. This data does not include clean records of flights with no accidents or incidents. When recommending an aircraft company, aircraft model, or location of operation, I used the basis of which has the lowest number of accidents/incidents, with zero accidents/incidents not being an option.

2. 

## Recommendations

This analysis has led me to conclude that the following three recommendations are: 
1. **Bombardier** is the aircraft manufacturer that I suggest we purchase. Bombardier has overall the least average number of injuries and the highest unijured rate when compared to Airbus, Boeing and Embraer.

2.  

## Next Steps

To further lower the risk of this new business venture, further analysis is required to consider:
1. **Cost** of aircraft and operation of such aircraft.

2. **Profit** based on location of operation.

3. Interrupted flight due to **weather** when choosing location.